# Chapter 1. The Python Data Model

One of the best qualities of Python is its consistency.

However, if you learned another object-oriented language before Python, you may find it strange to use `len(collection)` instead of `collection.len()`. This apparent oddity is the tip of an iceberg that, when properly understood, is the key to everything we call *Pythonic*. The iceberg is called the Python Data Model, and it is the API that we use to make our own objects play well with the most idiomatic language features.

You can think of the data model as a description of Python as a framework. It formalizes the interfaces of the building blocks of the language itself, such as sequences, functions, iterators, coroutines, classes, context managers, and so on.

The Python interpreter invokes special methods to perform basic object operations, often triggered by special syntax. The special method names are always written with leading and trailing double underscores. For example, the syntax `obj[key]` is supported by the `__getitem__` special method. In order to evaluate `my_collection[key]`, the interpreter calls `my_collection.__getitem__(key)`.

We implement special methods when we want our objects to support and interact with fundamental language constructs such as:

- Collections
- Attribute access
- Iteration (including asynchronous iteration using `async for`)
- Operator overloading
- Function and method invocation
- String representation and formatting
- Asynchronous programming using `await`
- Object creation and destruction
- Managed contexts using the `with` or `async with` statements

---

## A Pythonic Card Deck

A deck as a sequence of playing cards example. Is simple, but it demonstrates the power of implementing just two special methods, `__getitem__` and `__len__`.

In [1]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

The first thing to note is the use of `collections.namedtuple` to construct a simple class to represent individual cards. We use `namedtuple` to build classes of objects that are just bundles of attributes with no custom methods, like a database record.

In [2]:
beer_card = Card('7', 'diamonds')
beer_card

Card(rank='7', suit='diamonds')

It’s short, but it packs a punch. First, like any standard Python collection, a deck responds to the `len()` function by returning the number of cards in it:

In [3]:
deck = FrenchDeck()
len(deck)

52

Reading specific cards from the deck—say, the first or the last—is easy, thanks to the `__getitem__` method:

In [4]:
print(deck[0])
print(deck[-1])

Card(rank='2', suit='spades')
Card(rank='A', suit='hearts')


Should we create a method to pick a random card? No need. Python already has a function to get a random item from a sequence: `random.choice`. We can use it on a deck instance:

In [5]:
from random import choice

print(choice(deck))
print(choice(deck))
print(choice(deck))

Card(rank='Q', suit='spades')
Card(rank='4', suit='spades')
Card(rank='8', suit='hearts')


We’ve just seen two advantages of using special methods to leverage the Python Data Model:

- Users of your classes don’t have to memorize arbitrary method names for standard operations. (“How to get the number of items? Is it `.size()`, `.length()`, or what?”)
- It’s easier to benefit from the rich Python standard library and avoid reinventing the wheel, like the `random.choice` function.

But it gets better.

Because our `__getitem__` delegates to the `[]` operator of `self._cards`, our deck automatically supports slicing. Here’s how we look at the top three cards from a brand-new deck, and then pick just the aces by starting at index 12 and skipping 13 cards at a time:

In [ ]:
print(deck[:3])
print(deck[12::13])

Just by implementing the `__getitem__` special method, our deck is also iterable:

In [6]:
for card in deck[:3]:
   print(card)

Card(rank='2', suit='spades')
Card(rank='3', suit='spades')
Card(rank='4', suit='spades')


We can also iterate over the deck in reverse:

In [7]:
for card in reversed(deck[:3]):
   print(card)

Card(rank='4', suit='spades')
Card(rank='3', suit='spades')
Card(rank='2', suit='spades')


Iteration is often implicit. If a collection has no `__contains__` method, the `in` operator does a sequential scan. Case in point: `in` works with our `FrenchDeck` class because it is iterable. Check it out:

In [8]:
Card('Q', 'hearts') in deck

True

In [9]:
Card('7', 'beasts') in deck

False

How about sorting? A common system of ranking cards is by rank (with aces being highest), then by suit in the order of spades (highest), hearts, diamonds, and clubs (lowest). Here is a function that ranks cards by that rule, returning `0` for the 2 of clubs and `51` for the ace of spades.

Given `spades_high`, we can now list our deck in order of increasing rank:

In [15]:
suit_values = dict(spades=3, hearts=2, diamonds=1, clubs=0)

def spades_high(card):
    rank_value = FrenchDeck.ranks.index(card.rank)
    return rank_value * len(suit_values) + suit_values[card.suit]

display=0
for i, card in enumerate(sorted(deck, key=spades_high)):
    if i%5 ==0:
        print(card)

Card(rank='2', suit='clubs')
Card(rank='3', suit='diamonds')
Card(rank='4', suit='hearts')
Card(rank='5', suit='spades')
Card(rank='7', suit='clubs')
Card(rank='8', suit='diamonds')
Card(rank='9', suit='hearts')
Card(rank='10', suit='spades')
Card(rank='Q', suit='clubs')
Card(rank='K', suit='diamonds')
Card(rank='A', suit='hearts')


Although `FrenchDeck` implicitly inherits from the `object` class, most of its functionality is not inherited, but comes from leveraging the data model and composition. By implementing the special methods `__len__` and `__getitem__`, our `FrenchDeck` behaves like a standard Python sequence, allowing it to benefit from core language features (e.g., iteration and slicing) and from the standard library, as shown by the examples using `random.choice`, `reversed`, and `sorted`. Thanks to composition, the `__len__` and `__getitem__` implementations can delegate all the work to a `list` object, `self._cards`.